<a href="https://colab.research.google.com/github/kbakande/Hugging_Face_Library/blob/main/course/en/chapter7/section2_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Token classification (PyTorch)](https://huggingface.co/learn/nlp-course/en/chapter7/2?fw=pt)

* The Dataset structure contains rows of dictionary where each dictionary correspond to a data point consisting of features and coresponding values
* `Dataset` object also has a number of methods and attributes
* `DataCollatorForTokenClassification` pads with `-100`
* The traditional framework used to evaluate token classification prediction is [`seqeval`](https://huggingface.co/spaces/evaluate-metric/seqeval)

#### Main steps to prepare custom training loop
1. Prepare the dataloaders
2. Prepare the model -> model initialisation and passing in 1d2label and label2id
3. Prepare the optimizer
4. Pass all to accelerate.prepare()

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00
ERROR: torch_xla-1.9-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "koakande@gmail.com"
!git config --global user.name "koakande"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [7]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [8]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [9]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

##### Matching example tokens to their labels
* Each example in `conll2003` data have labels/tags for NER, POS and Chunking tasks
* It means each tokens/words in each example have labels
* To match words to labels:
  * extract all possible labels which are in a list
  * for each example, extract its labels which are numerical with the number corresponding to the position of the label in all possible labels list
  * Use the numerical number to extract the corresponding label from the all possible labels list.

In [10]:
  # the max length are added to pretty print line 1 and line 2 so the tokens have
#  same spacing as the labels when aligned as printed
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [11]:
pos_feature = raw_datasets["train"].features["pos_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

#### Parts of speech tagger
Adverb:

* RB: Adverb
* RBR: Comparative adverb (e.g., "faster", "sooner")
* RBS: Superlative adverb (e.g., "fastest", "soonest")

Particle:

* RP: Particle (e.g., "up", "off", "out")

Symbol:

* SYM: Symbol

To:

* TO: "to" as preposition or infinitive marker

Interjection:

* UH: Interjection (e.g., "oh", "wow", "ouch")

Verb:

* VB: Verb, base form
* VBD: Verb, past tense
* VBG: Verb, gerund or present participle
* VBN: Verb, past participle
* VBP: Verb, present tense, not 3rd person
singular
* VBZ: Verb, present tense, 3rd person singular

Wh-Determiner:

* WDT: Wh-determiner (e.g., "which", "what")

Wh-Pronoun:

* WP: Wh-pronoun (e.g., "who", "what", "whom")
* WP$: Possessive wh-pronoun (e.g., "whose")

Wh-Adverb:

* WRB: Wh-adverb (e.g., "how", "when", "where", "why")


#### Alternate part of speech tagger

* CC: Coordinating Conjunction
* CD: Cardinal Number
* DT: Determiner
* EX: Existential There
* FW: Foreign Word
* IN: Preposition or Subordinating Conjunction
* JJ: Adjective
* JJR: Adjective, Comparative
* JJS: Adjective, Superlative
* LS: List Item Marker
* MD: Modal
* NN: Noun, Singular or Mass
* NNP: Proper Noun, Singular
* NNPS: Proper Noun, Plural
* NNS: Noun, Plural
* NN|SYM: Noun or Symbol
* PDT: Predeterminer
* POS: Possessive Ending
* PRP: Personal Pronoun
* PRP: Possessive Pronoun
* RB: Adverb
* RBR: Adverb, Comparative
* RBS: Adverb, Superlative
* RP: Particle
* SYM: Symbol
* TO: to
* UH: Interjection
* VB: Verb, Base Form
* VBD: Verb, Past Tense
* VBG: Verb, Gerund or Present Participle
* VBN: Verb, Past Participle
* VBP: Verb, Non-3rd Person Singular Present
* VBZ: Verb, 3rd Person Singular Present
* WDT: Wh-Determiner
* WP: Wh-Pronoun
* WP: Possessive Wh-Pronoun
* WRB: Wh-Adverb

In [12]:
# the max length are added to pretty print line 1 and line 2 so the tokens have
#  same spacing as the labels when aligned as printed
pos_feature = raw_datasets["train"].features["pos_tags"]
pos_label_names = pos_feature.feature.names
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["pos_tags"]

line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = pos_label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU  rejects German call to boycott British lamb . 
NNP VBZ     JJ     NN   TO VB      JJ      NN   . 


In [13]:
# the max length are added to pretty print line 1 and line 2 so the tokens have
#  same spacing as the labels when aligned as printed
pos_feature = raw_datasets["train"].features["chunk_tags"]
pos_label_names = pos_feature.feature.names
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["chunk_tags"]

line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = pos_label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(pos_label_names)
print(line1)
print(line2)

['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP']
EU   rejects German call to   boycott British lamb . 
B-NP B-VP    B-NP   I-NP B-VP I-VP    B-NP    I-NP O 


#### Chunk menaings

O: Outside of any chunk
B-ADJP: Beginning of an Adjective Phrase
I-ADJP: Inside of an Adjective Phrase
B-ADVP: Beginning of an Adverb Phrase
I-ADVP: Inside of an Adverb Phrase
B-CONJP: Beginning of a Conjunction Phrase
I-CONJP: Inside of a Conjunction Phrase
B-INTJ: Beginning of an Interjection
I-INTJ: Inside of an Interjection
B-LST: Beginning of a List Marker
I-LST: Inside of a List Marker
B-NP: Beginning of a Noun Phrase
I-NP: Inside of a Noun Phrase
B-PP: Beginning of a Prepositional Phrase
I-PP: Inside of a Prepositional Phrase
B-PRT: Beginning of a Particle
I-PRT: Inside of a Particle
B-SBAR: Beginning of a Clause introduced by a Subordinating Conjunction
I-SBAR: Inside of a Clause introduced by a Subordinating Conjunction
B-UCP: Beginning of a Multi-Word Unit
I-UCP: Inside of a Multi-Word Unit
B-VP: Beginning of a Verb Phrase
I-VP: Inside of a Verb Phrase


In [14]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
tokenizer.is_fast

True

In [16]:
# adds is_split into words to warn the tokenizer object that input is
# already pretokenized into words
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [17]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [18]:
# expand labels to align with model input tokens
# this arises because the training data has pretokenized words
# with labels so when BERT tokenizer eventually tokenized the
# pretokenized words, they are more which cause a mismatch
#  between the BERT tokens and the training data labels
#  So labels needed expansion to match BERT tokens length for
#  proper fine tuning
def align_labels_with_tokens(labels, word_ids):
    null_label = -100
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = null_label if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(null_label)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [19]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


🖊 Some researchers prefer to attribute only one label per word, and assign -100 to the other subtokens in a given word. This is to avoid long words that split into lots of subtokens contributing heavily to the loss. Change the previous function to align labels with input IDs by following this rule.

In [20]:
# assign zero to other subtokens in a given word
def align_labels_with_zero_subtokens(labels, word_ids):
    null_label = -100
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = null_label if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(null_label)
        else:
            # Same word as previous token
            label = null_label
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [21]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_zero_subtokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, -100, 0, -100]


In [22]:
# we need to take in batched examples, tokenize the batch and
# expand the corresponding labels
def tokenize_and_align_labels(examples):
    # tokenized the batch
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    # extract all labels for the batch -> multiple labels due from multple examples
    all_labels = examples["ner_tags"]
    new_labels = []
    # iterate over each example and expand the label
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
        if i < 2:
          print(new_labels)

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [23]:
# only return the tokenized fields and labels
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

[[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]]
[[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]
[[-100, 7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 1, 2, 2, 0, 0, 0, 7, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 1, 2, 2, 0, 0, 0, 7, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 1, 2, 2, 2, 0, 5, 6, 0, 1, 2, 2, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

[[-100, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 5, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 7, 8, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 3, 4, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 3, 4, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 3, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 7, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 7, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]]


Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

[[-100, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 1, 2, 2, 2, 2, 2, -100]]
[[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]]
[[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 3, 4, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 3, 4, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 3, 4, 0, 0, 0, 0, 0, 0, 0, -100]]
[[-100, 3, 4, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 3, 0, 0, 0, 0, 0, 0, 0, -100]]


In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(batch["labels"])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])


In [26]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


* Comparing the above outputs shows that BERT/HuggingFace AutoTokenizerForTokenClassification pads with `-100`.

In [27]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 600.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=458ae6a84124c9dafcda81750e52050bbfd1edd750ebd40049b323f9b3dae547
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [28]:
# the metric seqeval is loaded via evaluate
import evaluate

metric = evaluate.load("seqeval")

In [29]:
label_names
labels

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [30]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [31]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

#### Model output
* Model logits matrix size is token_num by label_num. That is the number of tokens in input text by num of possible labels.
* This gives probability of each token being/belonging in any of the labelled entities
* np.argmax will then return the index of the label with the highest logits titled predictions
* compute metrics takes in a model output for an example and compute the metrics

In [32]:
import numpy as np


def compute_metrics(eval_preds):
    # obtain model output logits and corresponding labels
    logits, labels = eval_preds
    # extract index of max logit for each token
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    # compute metric per example/sample
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [33]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [34]:
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [35]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
model.config.num_labels

9

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

ValueError: ignored

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [40]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [41]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [43]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [44]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [45]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'koakande/bert-finetuned-ner-accelerate'

In [46]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/koakande/bert-finetuned-ner-accelerate into local empty directory.


In [47]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [48]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch 0: {'precision': 0.9301581958936385, 'recall': 0.8972402597402598, 'f1': 0.9134027433482068, 'accuracy': 0.9820303761700123}
epoch 1: {'precision': 0.9414338606529788, 'recall': 0.9217333992420498, 'f1': 0.9314794771459496, 'accuracy': 0.9844587037146053}
epoch 2: {'precision': 0.9474924267923258, 'recall': 0.9241628365068942, 'f1': 0.9356822336712648, 'accuracy': 0.9862541943839407}


In [49]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [50]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'entity_group': 'PER',
  'score': 0.9988506,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9647625,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9986118,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]